## 일베 댓글로 fast-text 임베딩 하기

In [ ]:
### 전처리 코드 블로그에서 받아온거
import re
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
targetXML = open('./ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행한다.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
### FastText 학습
from gensim.models import FastText
ft_model = FastText(result, size=100, window=5, min_count=5, workers=4, sg=1)

In [4]:
import pandas as pd
import numpy as np
import re

In [3]:
df = pd.read_csv('./비속어데이터셋_전처리o.csv')
df.head(10)

,댓글,비속어여부
0,좌배 까는건,1
1,집에 롱 패딩만 세 개다 10년 더 입어야지,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고,1
5,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,1
6,재앙이한건햇노,1
7,글쓴이 와꾸 승리에 비하면 방사능 피폭 원숭이 일듯,1
8,마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나,1
9,은행에 대출 상담 받으러 가보면 직업의 귀천 바로 알려줌,0


In [5]:
# 전체 데이터에 적용시키기 위한 과정

def preprocessing(sentence):
    sentence_split = sentence.split()
    sentence_split_result = []
    for word in sentence_split:
        if word[0] != '@' and word[:4] !='http':
            sentence_split_result.append(word)
    sentence_result = ' '.join(sentence_split_result)
    text_result = ''.join(re.compile('[가-힣|0-9|a-z| ]+').\
    findall(sentence_result)).strip()
    return text_result

df["댓글"] = df["댓글"].apply(preprocessing)


In [6]:
df.head(10)

,댓글,비속어여부
0,좌배 까는건,1
1,집에 롱 패딩만 세 개다 10년 더 입어야지,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고,1
5,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,1
6,재앙이한건햇노,1
7,글쓴이 와꾸 승리에 비하면 방사능 피폭 원숭이 일듯,1
8,마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나,1
9,은행에 대출 상담 받으러 가보면 직업의 귀천 바로 알려줌,0


In [7]:
# white space 데이터를 empty value로 변경

df["댓글"] = df["댓글"].str.replace('^ +', "")
df["댓글"].replace('', np.nan, inplace=True)
print(df.isnull().sum())

댓글       0
비속어여부    0
dtype: int64


In [9]:
# 문장 내 한글자만 이루어진 댓글은 삭제

def onewordremove(sentence):
    sentence_list = []
    for index in range(len(sentence.split())):
        if len(sentence.split()[index]) > 1: #문자 1글자 초과일 때
            sentence_list.append(sentence.split()[index]) #문자 1글자로 이루어진 단어만 빼고 sentence_list에 추가
    sentence = " ".join(sentence_list) #리스트의 내용을 하나의 문자열로 join
    return sentence

df['댓글'] = df['댓글'].apply(onewordremove)

In [13]:
df

,댓글,비속어여부
0,좌배 까는건,1
1,집에 패딩만 개다 10년 입어야지,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고,1
...,...,...
6764,후장꽂아,1
6765,후장뚫어,1
6766,흐접,1
6767,흐젚,1


In [14]:
# 불용어 제거
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다','부터']

In [15]:
tmp = df.copy()

# 자모 분류, fast text 는 git 참고 
https://github.com/smothly/bad-word-detection/blob/master/FastTextVocab.ipynb

In [15]:
df.dtypes

문장       object
혐오 여부     int64
글자수       int64
dtype: object

In [16]:
# 특수문자나 한자 등이 안날라 갔을 경우를 처리
import re
pattern = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]") # 한글 숫자 영어 공백 말고 제거

# 각 단어에 해당 해당 정규표현식 적용
def clear_word(word):
    word = re.sub(pattern, "", word)
    return word

df['문장'] = df['문장'].astype('str')
df['문장'] = df['문장'].apply(lambda x: clear_word(x))

In [17]:
# index 뒤죽박죽 된 것 재정비
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df = df[1:]

In [18]:

CHOSUNGS = [u'ㄱ',u'ㄲ',u'ㄴ',u'ㄷ',u'ㄸ',u'ㄹ',u'ㅁ',u'ㅂ',u'ㅃ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅉ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
JOONGSUNGS = [u'ㅏ',u'ㅐ',u'ㅑ',u'ㅒ',u'ㅓ',u'ㅔ',u'ㅕ',u'ㅖ',u'ㅗ',u'ㅘ',u'ㅙ',u'ㅚ',u'ㅛ',u'ㅜ',u'ㅝ',u'ㅞ',u'ㅟ',u'ㅠ',u'ㅡ',u'ㅢ',u'ㅣ']
JONGSUNGS = [u'_',u'ㄱ',u'ㄲ',u'ㄳ',u'ㄴ',u'ㄵ',u'ㄶ',u'ㄷ',u'ㄹ',u'ㄺ',u'ㄻ',u'ㄼ',u'ㄽ',u'ㄾ',u'ㄿ',u'ㅀ',u'ㅁ',u'ㅂ',u'ㅄ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
TOTAL = CHOSUNGS + JOONGSUNGS + JONGSUNGS

# 자모분리
def jamo_split(word, end_char="_"):
    
    result = []
    
    for char in word:
        
        character_code = ord(char)
        
        if 0xD7A3 < character_code or character_code < 0xAC00:
            result.append(char)
            continue

        chosung_index = int((((character_code - 0xAC00) / 28) / 21) % 19)
        joongsung_index = int(((character_code - 0xAC00) / 28) % 21)
        jongsung_index = int((character_code - 0xAC00) % 28)
        
        chosung = CHOSUNGS[chosung_index]
        joongsung = JOONGSUNGS[joongsung_index]
        jongsung = JONGSUNGS[jongsung_index]
        
        # 종성 범위 밖에 있는 것들은 end_char로 메꿔준다.
        if jongsung_index == 0:
            jongsung = end_char
        
        result.append(chosung)
        result.append(joongsung)
        result.append(jongsung)

    return "".join(result)


In [19]:
# 각 문장을 jamo_split하여 저장
#from JamoSplit import jamo_split, jamo_combine
tmp.loc[:,'댓글'] = tmp.loc[:,'댓글'].apply(lambda x: jamo_split(x))
tmp.loc[:,'댓글'] = tmp.loc[:,'댓글'].apply(lambda x: x.split(" "))

In [20]:
tmp.sample(15)

,댓글,비속어여부
2658,"[ㄱㅕㅇㅇㅟ_, ㅇㅣ_ㅅㅏㅇ, ㅅㅏ_ㅂㅓㅂㄱㅕㅇㅊㅏㄹㄱㅘㄴㅁㅏㄴ, ㄱㅏ_ㄴㅡㅇㅅㅜㄴ...",0
317,"[ㅇㅒ_ㄷㅗ_, ㅎㅓㄴㅂㅓㅂ, ㄱㅏㅇㅇㅢ_ㅎㅐ_ㅅㅓ_, ㅈㅗ_ㅁㅜ_ㅅㅏ_, ㄷㅚ_ㅁ...",0
4641,"[ㄱㅔ_ㅇㅣ_ㅇㅑ_, ㅇㅛㅇㄱㅣ_ㅅㅏ_ㅁㅕㄴ, ㄷㅡ_ㄹㅐ_ㄱㅗㄴ, ㅅㅏ_ㅇㅕㄱㅎㅏ_...",1
2594,"[ㄱㅡㄴㄷㅔ_, ㅇㅣㄹㄱㅔ_ㅇㅣ_ㄷㅡㄹㅇㅡㄴ, ㄱㅘㄴㄱㅖ_ㅇㅓㅄㅈㅏ_ㄴㅏ_, ㄱㅕㄹ...",1
5304,"[ㅈㅓ_ㄱㅓ_, ㄸㅏㄱㅂㅘ_ㄷㅗ_ㅉㅣㄴㄸㅏ_ㅅㅐ_ㄲㅣ_ㄱㅏ_, ㅇㅣㄹㅈㅣㄴㅎㅕㅇㄴㅣ...",1
692,"[ㅇㅠㄱㄱㅗ_ㄱㅣ_ㄴㅡㄴ, ㅁㅏㅅㅇㅣㅆㅇㅝ_]",0
4277,"[ㅇㅓ_ㅉㅓ_ㄹㅏ_ㄱㅗ_ㅎㅏ_ㅊㅏㄶㅇㅡㄴ, ㅊㅜㄱㅅㅐㅇㅎㅏㄴㅌㅔ_ㄲㅏ_ㅈㅣ_, ㄴㅏ...",0
3132,"[ㅅㅣㄴㅂㅏㄹㅁㅓ_ㅇㅣㅁ, ㄱㅐ_ㅇㅣ_ㅃㅡ_]",0
2624,"[ㅇㅜㅅㄱㅗ_ㄱㅏㄴㄷㅏ_, ㅂㅕㅇㅅㅣㄴㅇㅏ_]",1
3978,"[ㅂㅔ_ㅊㅜㅇㅇㅣ_ㅇㅘㄹㅇㅣ_ㄱㅓㅅㅇㅡㄴ, ㅈㅗ_ㅈㅏㄱㅇㅣ_ㅁㅡ_ㄴㅣ_ㄷㅏ_, ㅃㅏ...",1


In [21]:
# dataframe -> list로 변환
sentence_list = list(tmp['댓글'])
len(sentence_list)

6769

In [22]:
# fasttext 인풋 완성!
sentence_list[:10]

[['ㅈㅘ_ㅂㅐ_', 'ㄲㅏ_ㄴㅡㄴㄱㅓㄴ'],
 ['ㅈㅣㅂㅇㅔ_', 'ㅍㅐ_ㄷㅣㅇㅁㅏㄴ', 'ㄱㅐ_ㄷㅏ_', '10ㄴㅕㄴ', 'ㅇㅣㅂㅇㅓ_ㅇㅑ_ㅈㅣ_'],
 ['ㄱㅐ_ㅅㅗ_ㄹㅣ_ㅇㅑ_',
  'ㄴㅣ_ㄱㅏ_',
  'ㅃㅏㄹㄱㅐㅇㅇㅣ_ㄹㅡㄹ',
  'ㅇㅗㅇㅎㅗ_ㅎㅏ_ㄱㅗ_',
  'ㄷㅡ_ㄹㅜ_ㅋㅣㅇㅇㅡㄹ',
  'ㅈㅣㅅㅇㅣ_ㄹㅏ_ㄱㅗ_',
  'ㅁㅏㄹㅁㅗㅅㅎㅐ_ㅅㅓ_',
  'ㅃㅣ_ㅈㅣㄴㄱㅓ_ㅇㅑ_',
  'ㅃㅏㄹㄱㅐㅇㅇㅏ_'],
 ['ㅅㅔ_ㅌㅏㄱㅇㅣ_ㄹㅏ_ㄱㅗ_', 'ㅂㅘ_ㄷㅗ_', 'ㄷㅚㄴㄷㅏ_'],
 ['ㅇㅐ_ㅅㅐ_ㄲㅣ_ㄱㅏ_', 'ㅊㅗ_ㄷㅣㅇㄷㅗ_', 'ㅇㅏ_ㄴㅣ_ㄱㅗ_'],
 ['731ㅂㅜ_ㄷㅐ_ㅇㅢ_',
  'ㅎㅜ_ㅇㅖ_ㄹㅏ_',
  'ㄱㅡ_ㄹㅓㄴㅈㅣ_',
  'ㄱㅏ_ㅎㅏㄱㅈㅓㄱㅇㅣㄴ',
  'ㅇㅏ_ㅇㅣ_ㄷㅣ_ㅇㅓ_ㄴㅡㄴ',
  'ㅅㅔ_ㄱㅖ_ㅊㅚ_ㄱㅗ_ㅇㅣㅁ',
  'ㅇㅣ_ㄹㅐ_ㅅㅓ_',
  'ㅇㅐ_ㄱㅛ_ㅁㅏㄴ',
  'ㄸㅓㄹㅇㅓ_ㄷㅗ_',
  'ㅂㅓㄹㄹㅣ_ㄴㅡㄴ',
  'ㅎㅏㄴㄱㅜㄱㅇㅔ_',
  'ㄱㅣ_ㄹㅡㄹ',
  'ㅆㅓ_ㅅㅓ_',
  'ㅈㅣㄴㅊㅜㄹㅎㅏ_ㄹㅕ_ㄱㅗ_',
  'ㅎㅏ_ㅈㅣ_ㅈㅗ_ㅅㅔㄴㄴㅏㅁㅈㅏ_ㄷㅡㄹㅇㅡㄴ',
  'ㅇㅣ_ㅃㅡㄴㅇㅕ_ㅈㅏ_ㅁㅏㄴ',
  'ㅂㅗ_ㅁㅕㄴ',
  'ㅅㅏ_ㅈㅗㄱㅇㅡㄹ',
  'ㅁㅗㅅㅆㅡ_ㅁㅕ_',
  'ㄱㅗㅇㅈㅜ_ㄷㅐ_ㅈㅓㅂㅎㅐ_ㅈㅜ_ㄴㅡㄴ',
  'ㄴㅗㅁㄷㅡㄹㅇㅣ_ㄴㅣ_'],
 ['ㅈㅐ_ㅇㅏㅇㅇㅣ_ㅎㅏㄴㄱㅓㄴㅎㅐㅅㄴㅗ_'],
 ['ㄱㅡㄹㅆㅡㄴㅇㅣ_',
  'ㅇㅘ_ㄲㅜ_',
  'ㅅㅡㅇㄹㅣ_ㅇㅔ_',
  'ㅂㅣ_ㅎㅏ_ㅁㅕㄴ',
  'ㅂㅏㅇㅅㅏ_ㄴㅡㅇ',
  'ㅍㅣ_ㅍㅗㄱ',
  'ㅇㅝㄴㅅㅜㅇㅇㅣ_',
  'ㅇㅣㄹㄷㅡㅅ'],
 ['ㅆㅣ_ㅂㅏㄹㄹㅕㄴ',
  'ㅁㅕㅊㅍㅕㅇㅇㅣ_ㄱㅗ_',
  'ㅁㅐㅅㄱㅐ_ㄷㅡ_ㄱㅏㅆㄴㅗ_',
  'ㄷㅐ_ㅎㅏ_ㅇㅣ_ㅎㅐㅁㅎㅏ_ㄱㅗ_',
  'ㅎㅐ_ㅂㅏㅆㄴㅏ_'],
 ['ㅇㅡㄴㅎㅐㅇㅇㅔ_',
  'ㄷㅐ_ㅊㅜㄹ',
  'ㅅㅏㅇㄷㅏㅁ',
  'ㅂㅏㄷㅇㅡ_ㄹㅓ_',
  'ㄱㅏ_ㅂㅗ_ㅁㅕㄴ',
  

### FastText WordEmbedding

In [23]:
# fasttext 적용
from gensim.models import FastText
# 임베딩 차원: 50
# window size: 좌우 2단어 비속어는 좌우단어와 별로 연관이 없다고 판단...
# min_count: 최소 3번 등장한 단어들
# workers: -1 전부!!
# sg: skipgram이 더 성능이 좋기 때문
# min_n max_n : n-gram단위인데 한글자가 3글자라 최소 자모3개부터 최대 6개까지 ngram하기로 하였다. 1글자 ~ 2글자
# iter: 반복횟수 10
model = FastText(sentence_list, size=50, window=2, min_count=3, workers=4, sg=1, min_n=3, max_n=6)

In [24]:
# 총 1769개 단어로 vocab이 만들어졌다.
# len(model.wv)
len(model.wv.vocab)

1769

In [25]:
# 연도 같은 경우는 연도와 비슷한 단어들이 나온다.
model.wv.most_similar(jamo_split("2018년"))

[('ㄱㅘㄴㅎㅏㄴ', 0.9992398023605347),
 ('ㅈㅣ_ㅂㅏㅇㅇㅡㄴ', 0.9991357326507568),
 ('ㅍㅐ_ㄷㅣㅇㅇㅡㄴ', 0.999131441116333),
 ('ㄱㅣㅁㅆㅣ_', 0.999125599861145),
 ('ㅁㅐ_ㄴㅕㄴ', 0.9991162419319153),
 ('ㅍㅐㄱㅌㅡ_ㄹㅡㄹ', 0.9991050958633423),
 ('ㄷㅗㄴㅇㅡㄹ', 0.9991037845611572),
 ('ㅇㅣ_ㄹㅡㅁ', 0.9991020560264587),
 ('ㅂㅜ_ㅂㅜㄴㄷㅗ_', 0.9990983009338379),
 ('ㅁㅏ_ㅌㅣ_ㅊㅣ_', 0.9990958571434021)]

In [26]:
# 욕설 같은 경우는 비슷한 형태의 욕설이 나온다.
model.wv.most_similar(jamo_split("개새끼"))

[('ㄱㅐ_ㅅㅐ_ㄲㅣ_ㄷㅡㄹ', 0.9999813437461853),
 ('ㅇㅣ_ㅅㅐ_ㄲㅣ_', 0.999981164932251),
 ('ㅈㅓ_ㅅㅐ_ㄲㅣ_', 0.999980628490448),
 ('ㄱㅐ_ㅅㅐ_ㄲㅣ_ㄱㅏ_', 0.9999799728393555),
 ('ㅅㅐ_ㄲㅣ_', 0.9999778270721436),
 ('ㅂㅕㅇㅅㅣㄴㅅㅐ_ㄲㅣ_', 0.9999767541885376),
 ('ㅈㅝ_ㄹㅏ_ㅇㅣ_ㅅㅐ_ㄲㅣ_', 0.9999737739562988),
 ('ㅂㅠㅇㅅㅣㄴㅅㅐ_ㄲㅣ_', 0.9999736547470093),
 ('ㅅㅐ_ㄲㅣ_ㄴㅡㄴ', 0.9999733567237854),
 ('ㅇㅔㅁㅊㅏㅇㅅㅐ_ㄲㅣ_', 0.9999712109565735)]

In [27]:
model.wv.most_similar(jamo_split("시발"))

[('ㅆㅣ_ㅂㅏㄹ', 0.9999410510063171),
 ('ㅇㅏ_ㅈㅣㄱㄲㅏ_ㅈㅣ_', 0.9999346733093262),
 ('ㅇㅘㅆㄴㅡㄴㄷㅔ_', 0.9999276399612427),
 ('ㅅㅏ_ㄹㅏㅁㅇㅣ_', 0.9999276399612427),
 ('ㄱㅏ_ㄹㅡ_ㅊㅣ_ㄴㅡㄴ', 0.9999261498451233),
 ('ㅇㅣㅆㅇㅡㅁ', 0.9999252557754517),
 ('ㅂㅕㅇㅅㅣㄴㅇㅣ_', 0.9999247789382935),
 ('ㄲㅏ_ㅈㅣ_', 0.9999244213104248),
 ('ㅇㅣㅆㄴㅡㄴㅈㅣ_', 0.9999243021011353),
 ('ㄷㅐ_ㅎㅏㄴ', 0.999923825263977)]

In [28]:
# 시발점을 쳤더니.. 아주 관련없는 단어들이 뜬다. 뭐지?
model.wv.most_similar(jamo_split("시발점"))

[('ㅅㅣ_ㅂㅏㄹ', 0.9999171495437622),
 ('ㅇㅏㄶㄴㅡㄴ', 0.9999160766601562),
 ('ㅇㅣ_ㅂㅓㄴㅇㅔ_', 0.9999122023582458),
 ('ㄱㅓㅅㅇㅣ_', 0.9999105930328369),
 ('ㅂㅗㄴㅇㅣㄴㅇㅣ_', 0.9999102354049683),
 ('ㅇㅏㄶㄴㅡㄴㄷㅏ_', 0.9999076128005981),
 ('ㄴㅗㅁㅇㅣ_', 0.9999071955680847),
 ('ㅇㅣㅆㅇㅓ_ㅇㅛ_', 0.9999057054519653),
 ('ㅇㅏ_ㅊㅣㅁㅇㅔ_', 0.9999046325683594),
 ('ㄱㅏ_ㄹㅡ_ㅊㅣ_ㄴㅡㄴ', 0.9999037981033325)]

In [29]:
model.wv.most_similar(jamo_split("새끼"))

[('ㅅㅐ_ㄲㅣ_ㄷㅡㄹㄷㅗ_', 0.999988317489624),
 ('ㅅㅐ_ㄲㅣ_ㄷㅡㄹㅇㅏ_', 0.9999850392341614),
 ('ㅅㅐ_ㄲㅣ_ㄱㅏ_', 0.9999841451644897),
 ('ㅂㅕㅇㅅㅣㄴㅅㅐ_ㄲㅣ_', 0.9999820590019226),
 ('ㅅㅐ_ㄲㅣ_ㄷㅡㄹ', 0.9999803900718689),
 ('ㅅㅐ_ㄲㅣ_ㄴㅡㄴ', 0.9999796152114868),
 ('ㄱㅐ_ㅅㅐ_ㄲㅣ_', 0.9999777674674988),
 ('ㄱㅐ_ㅅㅐ_ㄲㅣ_ㄱㅏ_', 0.9999760389328003),
 ('ㅅㅐ_ㄲㅣ_ㄷㅡㄹㅇㅣ_', 0.9999759197235107),
 ('ㅎㅗ_ㄹㅗ_ㅅㅐ_ㄲㅣ_', 0.999975323677063)]

In [30]:
model.wv.most_similar(jamo_split("존나"))

[('ㅃㅜㄴㅇㅣ_ㅈㅣ_', 0.9999749660491943),
 ('ㅎㅏ_ㄴㅏ_ㄱㅏ_', 0.9999688863754272),
 ('ㅇㅣ_ㄹㅏㄴㅇㅣ_', 0.9999685287475586),
 ('ㅊㅏ_ㄱㅏ_', 0.9999672174453735),
 ('ㄴㅏ_ㄹㅏ_ㄱㅏ_', 0.9999667406082153),
 ('ㅎㅏ_ㄴㅏ_', 0.9999665021896362),
 ('ㅇㅣ_ㄹㅐ_ㅅㅓ_', 0.9999662041664124),
 ('ㅁㅏㄴㄴㅏ_ㅅㅓ_', 0.9999660849571228),
 ('ㄸㅏ_ㄹㅏ_', 0.9999660849571228),
 ('ㅇㅣ_ㄹㅓㄴㅅㅣㄱㅇㅡ_ㄹㅗ_', 0.999965250492096)]

In [31]:
model.wv.most_similar(jamo_split("지랄"))

[('ㅈㅣ_ㄹㅏㄹㅎㅏ_ㄴㅔ_', 0.9999675750732422),
 ('ㅈㅣ_ㄹㅏㄹㄷㅗ_', 0.9999597072601318),
 ('ㅈㅣ_ㅈㅣ_ㅇㅠㄹㅇㅣ_', 0.9999583959579468),
 ('ㅇㅓㅄㄷㅏ_ㄴㅡㄴ', 0.9999532103538513),
 ('ㅎㅏ_ㅈㅣ_', 0.9999531507492065),
 ('ㅂㅕㅇㅅㅣㄴㅇㅣ_', 0.9999526739120483),
 ('ㅎㅏ_ㅈㅣ_ㅁㅏㄴ', 0.9999514818191528),
 ('ㅈㅏ_ㅈㅣ_', 0.9999507069587708),
 ('ㅈㅣ_ㄷㅣ_ㄱㅔ_ㅇㅣ_', 0.9999504089355469),
 ('ㅅㅐㅇㄱㅏㄱㄷㅗ_', 0.9999502897262573)]

In [32]:
model.wv.most_similar(jamo_split("벌레"))

[('ㄱㅓㄹㄹㅔ_', 0.999894917011261),
 ('ㅎㅏ_ㄱㅓ_ㄴㅏ_', 0.9998724460601807),
 ('ㅇㅏ_ㅈㅜㅁㅁㅏ_', 0.999872088432312),
 ('ㅂㅏ_ㅋㅟ_ㅂㅓㄹㄹㅔ_', 0.9998716711997986),
 ('ㄷㅏ_ㄴㅣ_ㄴㅡㄴㄷㅔ_', 0.9998692274093628),
 ('ㅂㅓㄹㅇㅓ_ㅅㅓ_', 0.9998685717582703),
 ('ㅂㅓㄹㄹㅔ_ㄷㅡㄹ', 0.9998675584793091),
 ('ㅇㅏ_ㅈㅜ_', 0.9998667240142822),
 ('ㅇㅗ_ㅈㅣ_ㄱㅔ_', 0.9998656511306763),
 ('ㄷㅚ_ㄴㅡㄴㄷㅔ_', 0.9998652338981628)]

In [30]:
# embedding model 저장
model.save("./gensim_festtext.model")

### 자모 분리하려고 했던 코드식들.. 

In [17]:
tmp = df.copy()

In [18]:
!pip install jamo

In [19]:
!pip install jamos-toolkit

In [20]:
# jamo 패키지 사용해보기 예시

from jamo import h2j, j2hcj
sample_text = "가나다한글"
j2hcj(h2j(sample_text))

'ㄱㅏㄴㅏㄷㅏㅎㅏㄴㄱㅡㄹ'

In [21]:
# # 자모 스플릿 하는 코드

# In[54]:


CHOSUNGS = [u'ㄱ',u'ㄲ',u'ㄴ',u'ㄷ',u'ㄸ',u'ㄹ',u'ㅁ',u'ㅂ',u'ㅃ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅉ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
JOONGSUNGS = [u'ㅏ',u'ㅐ',u'ㅑ',u'ㅒ',u'ㅓ',u'ㅔ',u'ㅕ',u'ㅖ',u'ㅗ',u'ㅘ',u'ㅙ',u'ㅚ',u'ㅛ',u'ㅜ',u'ㅝ',u'ㅞ',u'ㅟ',u'ㅠ',u'ㅡ',u'ㅢ',u'ㅣ']
JONGSUNGS = [u'_',u'ㄱ',u'ㄲ',u'ㄳ',u'ㄴ',u'ㄵ',u'ㄶ',u'ㄷ',u'ㄹ',u'ㄺ',u'ㄻ',u'ㄼ',u'ㄽ',u'ㄾ',u'ㄿ',u'ㅀ',u'ㅁ',u'ㅂ',u'ㅄ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
TOTAL = CHOSUNGS + JOONGSUNGS + JONGSUNGS

# 자모분리
def jamo_split(word, end_char="_"):
    
    result = []
    
    for char in word:
        
        character_code = ord(char)
        
        if 0xD7A3 < character_code or character_code < 0xAC00:
            result.append(char)
            continue

        chosung_index = int((((character_code - 0xAC00) / 28) / 21) % 19)
        joongsung_index = int(((character_code - 0xAC00) / 28) % 21)
        jongsung_index = int((character_code - 0xAC00) % 28)
        
        chosung = CHOSUNGS[chosung_index]
        joongsung = JOONGSUNGS[joongsung_index]
        jongsung = JONGSUNGS[jongsung_index]
        
        # 종성 범위 밖에 있는 것들은 end_char로 메꿔준다.
        if jongsung_index == 0:
            jongsung = end_char
        
        result.append(chosung)
        result.append(joongsung)
        result.append(jongsung)

    return "".join(result)


# 자모결합
def jamo_combine(word):

    result = ""
    index = 0
    
    while index < len(word):
    
        # 3개의 char를 보아 글자가 만들어지면 만들고 아니면 1개의 char만 추가한다.
        try:
            cho = CHOSUNGS.index(word[index]) * 21 * 28
            joong = JOONGSUNGS.index(word[index+1]) * 28
            jong = JONGSUNGS.index(word[index+2])

            result += chr(cho + joong + jong + 0xAC00) 
            index += 3

        except:
            result += word[index]
            index += 1
        
    return result

In [27]:
tmp.apply(to_be_divided)

AttributeError: 'Series' object has no attribute 'strip'

In [28]:
tmp.dtypes

문장       object
혐오 여부     int64
글자수       int64
dtype: object

In [111]:
# 한글 음절을 넘파이 배열로 바꾸기
# chr() 함수는 유니코드 번호를 받아서 해당하는 문자를 반환하는 함수 --> '가'에 해당하는 유니코드인 44032를 chr()에 넣으면 '가'가 반환
# ord() 함수는 문자를 받아서 대응하는 유니코드 번호를 반환 --> ord('가')를 치면 44032가 반환



syllables = np.array([chr(code) for code in range(44032, 55204)])
syllables = syllables.reshape(19, 21, 28)

# .... 이 패키지는 말을 직접 언급해줘야 하나보다 ㅜ

In [ ]:
초성의 인덱스 : (문자 - 0xAC00) / 21 / 28
중성의 인덱스 : (문자 - 0xAC00 - (초성 인덱스 * 21 * 28)) / 28 
종성의 인덱스 : (문자 - 0xAC00 - (초성 인덱스 * 21 * 28) - (중성 인덱스 * 28))

In [16]:
# 한글 초·중·종성 분리 --> 일부러 오타를 내어 비속어를 변형한 경우에도 비속어를 판별 할 수 있도록 한다


def run(x):
    consonant_list = [ord(char) for char in "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"] #초성 유니코드 리스트
    choseong_list = [char for char in "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"] #초성리스트
    jungseong_list = [char for char in "ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ"] #중성 리스트
    jongseong_list = [char for char in "-ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ"] #종성 리스트
    result = []
    
    for char in x:
        if ord(char)==32: #띄어쓰기인 경우
            result.append(char)
            
        elif 48<=ord(char)<=57: #숫자인 경우
            result.append(char)
            
        elif consonant_list.count(char) == 0:
            character_code = ord(char)
            
        if (55203 < character_code or character_code < 44032):
            continue 
            
            code = 44032
            choseong_index = (character_code - code) // 21 // 28
            jungseong_index = (character_code - code - (choseong_index * 21 *  28)) // 28
            jongseong_index = character_code - code - (choseong_index * 21 * 28) - (jungseong_index * 28)
            result.append(choseong_list[choseong_index])
            result.append(jungseong_list[jungseong_index])
            result.append(jongseong_list[jongseong_index])
            
        else:
            choseong_index = consonant_list.index(ord(char))
            result.append(choseong_list[choseong_index])
            result.append("-")
            result.append("-")
        return "".join(result)


In [17]:
# 각 문장을 jamo_split하여 저장
#from JamoSplit import jamo_split, jamo_combine
tmp.loc[:,'댓글'] = tmp.loc[:,'댓글'].apply(lambda x: run(x))
tmp.loc[:,'댓글'] = tmp.loc[:,'댓글'].apply(lambda x: x.split(" "))

ValueError: 51340 is not in list

In [122]:
class JamosSeparator():
    def __init__(self, string):
        self.string = string
        self.result = []
        self.choseong_list = [char for char in "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"]
        self.jungseong_list = [char for char in "ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ"]
        self.jongseong_list = [char for char in " ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ"]

    def run(self):
        for char in self.string:

            character_code = ord(char)

            # Do not process unless it is in Hangul syllables range.
            if 0xD7A3 < character_code or character_code < 0xAC00:
                continue

            choseong_index = (character_code - 0xAC00) // 21 // 28
            jungseong_index = (character_code - 0xAC00 - (choseong_index * 21 * 28)) // 28
            jongseong_index = character_code - 0xAC00 - (choseong_index * 21 * 28) - (jungseong_index * 28)

            self.result.append(self.choseong_list[choseong_index])
            self.result.append(self.jungseong_list[jungseong_index])
            self.result.append(self.jongseong_list[jongseong_index])
            self.result.append("_")

    def get(self):
        return self.result

In [26]:
def to_be_divided(korean_word):
    """
    한글 단어를 입력받아서 초성/중성/종성을 구분하여 리턴해줍니다. 
    """
    ####################################
    # 초성 리스트. 00 ~ 18
    CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ',
                    'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    # 중성 리스트. 00 ~ 20
    JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ',
                     'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    # 종성 리스트. 00 ~ 27 + 1(1개 없음)
    JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ',
                     'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    ####################################
    r_lst = []
    for w in list(korean_word.strip()):
        if '가'<=w<='힣':
            ch1 = (ord(w) - ord('가'))//588
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst

In [107]:
class JamoUtils:
    CHOSUNG = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ',
                    'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    JUNGSUNG = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ',
                    'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    JONGSUNG = ['', 'ㄱ', 'ㄲ', 'ᆪ', 'ᆫ', 'ᆬ', 'ᆭ', 'ㄷ',
                    'ㄹ', 'ᆰ', 'ᆱ', 'ᆲ', 'ᆳ', 'ᆴ', 'ᆵ', 'ᆶ', 'ㅁ', 'ㅂ', 'ᆹ', 'ᆺ', 'ᆻ', 'ᆼ',
                    'ᆽ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

#     def __init__(self):
#         raise NotImplementedError

    @staticmethod
    def split_one(target):
        code_point = ord(target)
        if 0xAC00 <= code_point <= 0xD79D:
            start_value = ord(target[0]) - 0xAC00
            jong = start_value % 28
            jung = int(((start_value - jong) / 28) % 21)
            cho = int((((start_value - jong) / 28) - jung) / 21)

            return JamoUtils.CHOSUNG[cho], JamoUtils.JUNGSUNG[jung], JamoUtils.JONGSUNG[jong]

        return target, '', ''

    @staticmethod
    def split(target):
        result = []
        for c in target:
            result.append(JamoUtils.split_one(c))

        return result

In [ ]:
syllables = np.array([chr(code) for code in range(44032, 55204)])
syllables = syllables.reshape(19, 21, 28)

In [142]:
# 내가 바꾼 코드

def run(x):
    consonant_list = [ord(char) for char in "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"] #초성 유니코드 리스트
    choseong_list = [char for char in "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"] #초성리스트
    jungseong_list = [char for char in "ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ"] #중성 리스트
    jongseong_list = [char for char in "-ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ"] #종성 리스트
    result = []
    
    for char in x:
        if ord(char)==32: #띄어쓰기인 경우
            result.append(char)
            
        elif 48<=ord(char)<=57: #숫자인 경우
            result.append(char)
            
        elif consonant_list.count(char) == 0:
            character_code = ord(char)
            
        if 0xD7A3 < character_code or character_code < 0xAC00:
            continue
            
            choseong_index = (character_code - 0xAC00) // 21 // 28
            jungseong_index = (character_code - 0xAC00 - (choseong_index * 21 * 28)) // 28
            jongseong_index = character_code - 0xAC00 - (choseong_index * 21 * 28) - (jungseong_index * 28)
            
            result.append(choseong_list[choseong_index])
            result.append(jungseong_list[jungseong_index])
            result.append(jongseong_list[jongseong_index])
            
        else:
            choseong_index = consonant_list.index(ord(char))
            result.append(choseong_list[choseong_index])
            result.append("-")
            result.append("-")
        return "".join(result)


In [168]:
# 내가 고친것 2

def to_be_divided(korean_word):
    """
    한글 단어를 입력받아서 초성/중성/종성을 구분하여 리턴해줍니다. 
    """
    # 초성 유니코드 리스트
    consonant_list = [ord(char) for char in "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"] #초성 유니코드 리스트
    
    ####################################
    # 초성 리스트. 00 ~ 18
    choseong_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ',
                    'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    # 중성 리스트. 00 ~ 20
    jungseong_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ',
                     'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    # 종성 리스트. 00 ~ 27 + 1(1개 없음)
    jongseong_list = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ',
                     'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    ####################################
    
    r_lst = []
    for char in list(korean_word.strip()):
        if '가'<=char<='힣':
            ch1 = (ord(char) - ord('가')) // 588
            ch2 = ((ord(char) - ord('가')) - (588 * ch1)) // 28
            ch3 = (ord(char) - ord('가')) - (588 * ch1) - ch2 * 28
            
            r_lst.append(choseong_list[ch1])
            r_lst.append(jungseong_list[ch2])
            r_lst.append(jongseong_list[ch3])
            
        else:
            choseong_index = consonant_list.index(ord(char))
            r_lst.append(choseong_list[choseong_index])
            r_lst.append("-")
            r_lst.append("-")
        return "".join(r_lst)


In [152]:
tmp7 = df.copy()

In [169]:
tmp7['문장'] = tmp7['문장'].apply(to_be_divided)

ValueError: 49 is not in list

In [165]:
ord(ss)

NameError: name 'ss' is not defined

In [170]:
ord('정')

51221

In [ ]:
import re
from konlpy.tag import Okt
from jamo import h2j, j2hcj #초성/중성/종성분리


f = open("./stopwords-ko.txt", 'r', encoding="utf-8")
lines = f.readlines()
stopwords = []
for line in lines:
    line = line.replace('\n', '')
    stopwords.append(line)
f.close()   

class c_token:    
# 불용어 파일 불러오기 ==> 밖에서 지정해저야 오류 안생김                         
         
# 1 + 2 + stopword 그냥 다 합치기!
    def preprocessing(self, input_str):
    # 문자만 필터링
        re_string = re.compile('[^ A-Za-z0-9ㄱ-ㅣ가-힣]+')\
            .sub('', input_str)
    #형태소단위 
        okt = Okt() #선택이유는 논문 참고
        m_token = okt.morphs(re_string, stem=True)                                
    # 초중성분리  ==> 너무 작게 쪼개져서 주석처리!!
        # cm_token = []
        # for i in m_token:
        #     cm_token.append( j2hcj(h2j(i)) )
    # 불용어 처리
        result = [token for token in m_token if token not in stopwords]
        preprocessed_text= ' '.join(result)
        
        return preprocessed_text




# 1.문자만 필터링(이모티콘제외) & stopword(불용어)
    def filter_str(self, input_str):
        re_string = re.compile('[^ A-Za-z0-9ㄱ-ㅣ가-힣]+').sub('', input_str)
        return print(re_string)

# 2. 
# 2-1.텍스트를 형태소 단위로 나눔
# norm(정규화),stem(어간추출) 기본값 False
    def m_tokenize(self, input_str):
        okt = Okt()
        m_token = okt.morphs(self.filter_str(input_str))
        return print(m_token)
#2-2. 초성/중성/종성분리
    def c_tokenize(self, input_str):
        c_token = j2hcj(h2j(self.filter_str(input_str)))
        return list(c_token)

# 형태소단위분리 & 초중종성분리
    def cm_tokenize(self, input_str):
        okt = Okt()
        mm_token = okt.morphs(self.filter_str(input_str))
        cm_token = []
        for i in mm_token:
            cm_token.append(self.c_tokenize(i))
        return cm_token

# 4.텍스트에서 명사만 ==> 워드클라우드 
# join(형태소/품사 설명) 기본값 False
    def noun_tokenize(self, input_str):
        okt = Okt()
        token = okt.nouns(self.filter_str(input_str))
        return token

# 테스트하기
# cl = c_token()
# test = "제품은 좋을수도 있고 나쁠수도 있는데 그걸 고객들을 속이려고 하고 심지어 사과도 안하고 양심도없지"
# result = cl.preprocessing(test)
# print(result)
# # ==> 제품 좋다 있다 나쁘다 있다 그걸 고객 속이다 하다 심지어 사과 하고 양심 없다

In [3]:
import pandas as pd
ddf = pd.read_csv('./hate_speech_data_womad.csv', index_col=0)
ddf.shape

(2000, 2)

In [4]:
ddf

,문장,혐오 여부
0,지금 어디 계세요?,0
1,한국 시간에 시계 맞췄어?,0
2,햄버거 두 개랑 콜라 주세요.,0
3,우리는 밤새 춤추고 노래했다.,0
4,"무엇보다도, 스트레스를 줄이는 것이 중요합니다.",0
...,...,...
1995,개독냄새 나노 ㅋㅋㅋ조팔 개오글,1
1996,사실상 페미니즘 영업사원 아니노 ㅋㅋㅋㅋ,1
1997,ㅋㅋㅋㅋㅋㅋㅋ,1
1998,아버지는 아침 일찍 서울로 가셨다.,0
